<a href="https://colab.research.google.com/github/eveapp/data_science/blob/ketki/Topic%20Classification%20F20/Reddit_Text_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, we clean and preprocess our reddit data.

For cleaning we do the following to the values of 'body' column:
  - Remove punctuations
  - Convert to lowercase
  - Remove Word Contractions (e.g. I've to I have).
  - Remove Stop Words (e.g. has, at, etc)

For Text Preprocessing we do the following to the values of 'cleaned_body' column:
 - Tokenize the text
 - Tag Parts of speech using NLTK standard POS tagging
 - Convert std NLTK POS tagging to WordNet POS tagging format. 
 - Lemmatize the Wordnet POS tagged data

 We save the cleaned and Preprocessed data into a csv for further EDA


In [ ]:
pip install contractions

     |████████████████████████████████| 245kB 7.8MB/s 
     |████████████████████████████████| 317kB 13.1MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81707 sha256=8240017205bb758d1282f4dfd588994b25e985dcd86b8b91704e9eb8f265cc10
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [ ]:
import contractions
import nltk
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
import string
%matplotlib inline

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
df_old=pd.read_csv('Labelled_dataset.csv')
df_old.head()

,Unnamed: 0,score,id,url,comms_num,title,body,timestamp,label,Sentiment,wordnet_pos,lemmatized,clean_body,clean_body_str,tokenized,pos_tags
0,NaN,88,d3mxw2,https://www.reddit.com/r/cancer/comments/d3mxw...,32,Fuck that Bell!,"Sorry for the title, but I wish the bell ringi...",2019-09-13 17:55:58,6.0,NaN,"[('sorry', 'a'), ('title', 'n'), ('wish', 'a')...","['sorry', 'title', 'wish', 'bell', 'ring', 'pa...","['sorry', 'title', 'wish', 'bell', 'ringing', ...",sorry title wish bell ringing party music cele...,"['sorry', 'title', 'wish', 'bell', 'ringing', ...","[('sorry', 'JJ'), ('title', 'NN'), ('wish', 'J..."
1,NaN,7,hesliu,https://www.reddit.com/r/CancerFamilySupport/c...,4,Father suffering from end stage aggression. Wo...,My dad was diagnosed with stage 4 bone cancer ...,2020-06-24 10:35:34,7.0,NaN,"[('dad', 'n'), ('diagnosed', 'v'), ('stage', '...","['dad', 'diagnose', 'stage', 'bone', 'cancer',...","['dad', 'diagnosed', 'stage', 'bone', 'cancer'...",dad diagnosed stage bone cancer january made d...,"['dad', 'diagnosed', 'stage', 'bone', 'cancer'...","[('dad', 'NN'), ('diagnosed', 'VBD'), ('stage'..."
2,NaN,76,50p82p,https://www.reddit.com/r/cancer/comments/50p82...,17,last chemo was today :),just finished my last of 12 abvd infusions. I ...,2016-09-02 4:01:24,2.0,NaN,"[('finished', 'v'), ('last', 'a'), ('abvd', 'a...","['finish', 'last', 'abvd', 'infusion', 'feel',...","['finished', 'last', 'abvd', 'infusions', 'fee...",finished last abvd infusions feel terrible kno...,"['finished', 'last', 'abvd', 'infusions', 'fee...","[('finished', 'VBN'), ('last', 'JJ'), ('abvd',..."
3,NaN,2,dg7uwq,https://www.reddit.com/r/CancerCaregivers/comm...,2,ER phobia?,Have any of your loved ones checked out of the...,2019-10-11 9:22:41,7.0,NaN,"[('loved', 'v'), ('ones', 'n'), ('checked', 'v...","['love', 'one', 'check', 'hospital', 'ama', 'm...","['loved', 'ones', 'checked', 'hospital', 'ama'...",loved ones checked hospital ama medical advice...,"['loved', 'ones', 'checked', 'hospital', 'ama'...","[('loved', 'VBN'), ('ones', 'NNS'), ('checked'..."
4,NaN,4,bry6ro,https://www.reddit.com/r/CancerCaregivers/comm...,10,"8 minutes into his appointment, another scene ...",At this point it’s just funny to me. I’m not t...,2019-05-23 11:16:30,3.0,NaN,"[('point', 'n'), ('it', 'n'), ('is', 'v'), ('f...","['point', 'it', 'be', 'funny', 'I', 'be', 'one...","['point', 'it is', 'funny', 'I am', 'one', 'th...",point it is funny I am one throwing temper tan...,"['point', 'it', 'is', 'funny', 'I', 'am', 'one...","[('point', 'NN'), ('it', 'PRP'), ('is', 'VBZ')..."


In [ ]:
df = df_old[['title', 'body', 'label']].copy()

In [ ]:
#(#Rows, #Columns)
df.shape

(2309, 3)

In [ ]:
df=df.dropna()
df.head()

,title,body,label
0,Fuck that Bell!,"Sorry for the title, but I wish the bell ringi...",6.0
1,Father suffering from end stage aggression. Wo...,My dad was diagnosed with stage 4 bone cancer ...,7.0
2,last chemo was today :),just finished my last of 12 abvd infusions. I ...,2.0
3,ER phobia?,Have any of your loved ones checked out of the...,7.0
4,"8 minutes into his appointment, another scene ...",At this point it’s just funny to me. I’m not t...,3.0


In [ ]:
#Rows that have values we could use
df.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 704 entries, 0 to 924
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   title   704 non-null    object 
 1   body    704 non-null    object 
 2   label   704 non-null    float64
dtypes: float64(1), object(2)
memory usage: 22.0+ KB


In [ ]:
#Detect Language of each 
# pretrained_model = "lid.176.bin" 
# model = fasttext.load_model(pretrained_model)
# langs = []
# for sent in rws['rating_description_str']:
#     lang = model.predict(sent)[0]
#     langs.append(str(lang)[11:13])
# rws['langs'] = langs

In [ ]:
#Make everything lowercase
df['clean_body'] =  df["title"].str.lower()+" "+ df["body"].str.lower() 

In [ ]:
def remove_punctuations_numbers(text):
    punc = string.punctuation 
    for punctuation in punc:
        text = text.replace(punctuation, '')
    return text

In [ ]:
def remove_stuff(text,remove_arr, replace_str):
    for rem in remove_arr:
        text = text.replace(rem, replace_str)
    return text

In [ ]:
import re 
def remove_digits(text):
  text=re.sub('[0-9]+', "number", text)
  return text

In [ ]:
#Replace all days of week by string "dayofweek"
weekstring =['monday', 'tuesday', 'wednesday', 'thursday', 'friday','saturday','sunday']
#Replace all month names by string "month"
months = ['january','february', 'march','april','may','june','july','august', 'september', 'october','november','december']
#Replace all the numbers by string "number"
nums=['one','two','three','four','five','six','seven','eight','nine','ten']
#Replace all family relations with string "famrel"
famrel=['dad','mum','daughter','son','aunt','uncle','husband','wife', 'brother','sister', 'father','mother']
#Replace all friendship (non-family) relations with string "friend"
friendrel=['boyfriend','girlfriend','colleague']

In [ ]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [ ]:
#remove punctuation 
df["clean_body"] = df["clean_body"].astype(str)
df['clean_body'] = df['clean_body'].apply(remove_punctuations_numbers)
df['clean_body'] = df['clean_body'].apply(remove_stuff, args=[famrel,'famrel'])
df['clean_body'] = df['clean_body'].apply(remove_stuff, args=[nums,'number'])
df['clean_body'] = df['clean_body'].apply(remove_stuff, args=[months,'month'])
df['clean_body'] = df['clean_body'].apply(remove_stuff, args=[friendrel,'friend'])
df['clean_body'] = df['clean_body'].apply(remove_stuff, args=[weekstring,'dayofweek'])
df['clean_body'] = df['clean_body'].apply(remove_digits)
df.head()

,title,body,label,clean_body
0,Fuck that Bell!,"Sorry for the title, but I wish the bell ringi...",6.0,fuck that bell sorry for the title but i wish ...
1,Father suffering from end stage aggression. Wo...,My dad was diagnosed with stage 4 bone cancer ...,7.0,famrel suffering from end stage aggression won...
2,last chemo was today :),just finished my last of 12 abvd infusions. I ...,2.0,last chemo was today just finished my last of...
3,ER phobia?,Have any of your loved ones checked out of the...,7.0,er phobia have any of your loved numbers check...
4,"8 minutes into his appointment, another scene ...",At this point it’s just funny to me. I’m not t...,3.0,number minutes into his appointment another sc...


In [ ]:
#Remove Word contractions
df['clean_body'] = df['clean_body'].apply(lambda x: [contractions.fix(word) for word in x.split()])
df.head()

,title,body,label,clean_body
0,Fuck that Bell!,"Sorry for the title, but I wish the bell ringi...",6.0,"[fuck, that, bell, sorry, for, the, title, but..."
1,Father suffering from end stage aggression. Wo...,My dad was diagnosed with stage 4 bone cancer ...,7.0,"[famrel, suffering, from, end, stage, aggressi..."
2,last chemo was today :),just finished my last of 12 abvd infusions. I ...,2.0,"[last, chemo, was, today, just, finished, my, ..."
3,ER phobia?,Have any of your loved ones checked out of the...,7.0,"[er, phobia, have, any, of, your, loved, numbe..."
4,"8 minutes into his appointment, another scene ...",At this point it’s just funny to me. I’m not t...,3.0,"[number, minutes, into, his, appointment, anot..."


In [ ]:
#Remove Stop Words
#stop_words = set(stopwords.words('english'))
stop_words1=[]
with open('stopwords.txt','r') as f:
  for line in f:
    for word in line.split():
      stop_words1.append(word.lower()) 
      #print(word)

#print(stopwords.words('english'))

print(len(stop_words1))
df['clean_body'] = df['clean_body'].apply(lambda x: [word for word in x if word not in stop_words1])
df.head()

94


,title,body,label,clean_body
0,Fuck that Bell!,"Sorry for the title, but I wish the bell ringi...",6.0,"[fuck, bell, sorry, title, wish, bell, ringing..."
1,Father suffering from end stage aggression. Wo...,My dad was diagnosed with stage 4 bone cancer ...,7.0,"[famrel, suffering, end, stage, aggression, wi..."
2,last chemo was today :),just finished my last of 12 abvd infusions. I ...,2.0,"[last, chemo, today, just, finished, last, num..."
3,ER phobia?,Have any of your loved ones checked out of the...,7.0,"[er, phobia, any, loved, numbers, checked, out..."
4,"8 minutes into his appointment, another scene ...",At this point it’s just funny to me. I’m not t...,3.0,"[number, minutes, appointment, another, scene,..."


In [ ]:
df.head(100)

,title,body,label,clean_body
0,Fuck that Bell!,"Sorry for the title, but I wish the bell ringi...",6.0,"[fuck, bell, sorry, title, wish, bell, ringing..."
1,Father suffering from end stage aggression. Wo...,My dad was diagnosed with stage 4 bone cancer ...,7.0,"[famrel, suffering, end, stage, aggression, wi..."
2,last chemo was today :),just finished my last of 12 abvd infusions. I ...,2.0,"[last, chemo, today, just, finished, last, num..."
3,ER phobia?,Have any of your loved ones checked out of the...,7.0,"[er, phobia, any, loved, numbers, checked, out..."
4,"8 minutes into his appointment, another scene ...",At this point it’s just funny to me. I’m not t...,3.0,"[number, minutes, appointment, another, scene,..."
...,...,...,...,...
96,Should I quit my job?,"Hi guys. \n\nIn early July, my mother was diag...",7.0,"[should, quit, job, guys, early, month, famrel..."
97,Losing hair...and time,My mother is losing her hair and has been give...,6.0,"[losing, hairand, time, famrel, losing, hair, ..."
98,My mom's first appointment,Last week my mom went to the doctor for a pers...,0.0,"[moms, first, appointment, last, week, mom, we..."
99,How do you cope with a loved one that is terminal,"Hi all,\n\nFirst post so please excuse me if I...",7.0,"[how, do, cope, loved, number, terminal, all, ..."


In [ ]:
#Clean body Str:
df['clean_body_str'] = [' '.join(map(str, l)) for l in df['clean_body']]
df.head()

,title,body,label,clean_body,clean_body_str
0,Fuck that Bell!,"Sorry for the title, but I wish the bell ringi...",6.0,"[fuck, bell, sorry, title, wish, bell, ringing...",fuck bell sorry title wish bell ringing party ...
1,Father suffering from end stage aggression. Wo...,My dad was diagnosed with stage 4 bone cancer ...,7.0,"[famrel, suffering, end, stage, aggression, wi...",famrel suffering end stage aggression will not...
2,last chemo was today :),just finished my last of 12 abvd infusions. I ...,2.0,"[last, chemo, today, just, finished, last, num...",last chemo today just finished last number abv...
3,ER phobia?,Have any of your loved ones checked out of the...,7.0,"[er, phobia, any, loved, numbers, checked, out...",er phobia any loved numbers checked out hospit...
4,"8 minutes into his appointment, another scene ...",At this point it’s just funny to me. I’m not t...,3.0,"[number, minutes, appointment, another, scene,...",number minutes appointment another scene made ...


In [ ]:
df.to_csv('EAFC_Raw.csv')

In [ ]:
#Tokenize
df['tokenized'] = df['clean_body_str'].apply(word_tokenize)
df.head()

,title,body,label,clean_body,clean_body_str,tokenized
0,Fuck that Bell!,"Sorry for the title, but I wish the bell ringi...",6.0,"[fuck, bell, sorry, title, wish, bell, ringing...",fuck bell sorry title wish bell ringing party ...,"[fuck, bell, sorry, title, wish, bell, ringing..."
1,Father suffering from end stage aggression. Wo...,My dad was diagnosed with stage 4 bone cancer ...,7.0,"[suffering, end, stage, aggression, will not, ...",suffering end stage aggression will not take p...,"[suffering, end, stage, aggression, will, not,..."
2,last chemo was today :),just finished my last of 12 abvd infusions. I ...,9.0,"[last, chemo, today, just, finished, last, abv...",last chemo today just finished last abvd infus...,"[last, chemo, today, just, finished, last, abv..."
3,ER phobia?,Have any of your loved ones checked out of the...,7.0,"[er, phobia, any, ones, checked, hospital, ama...",er phobia any ones checked hospital ama agains...,"[er, phobia, any, ones, checked, hospital, ama..."
4,"8 minutes into his appointment, another scene ...",At this point it’s just funny to me. I’m not t...,3.0,"[minutes, appointment, scene, made, plus, hosp...",minutes appointment scene made plus hospital h...,"[minutes, appointment, scene, made, plus, hosp..."


In [ ]:
#Tagging Parts of Speech in the tokenized 
df['pos_tags'] = df['tokenized'].apply(nltk.tag.pos_tag)
df.head()

,title,body,label,clean_body,clean_body_str,tokenized,pos_tags
0,Fuck that Bell!,"Sorry for the title, but I wish the bell ringi...",6.0,"[fuck, bell, sorry, title, wish, bell, ringing...",fuck bell sorry title wish bell ringing party ...,"[fuck, bell, sorry, title, wish, bell, ringing...","[(fuck, NN), (bell, NN), (sorry, JJ), (title, ..."
1,Father suffering from end stage aggression. Wo...,My dad was diagnosed with stage 4 bone cancer ...,7.0,"[suffering, end, stage, aggression, will not, ...",suffering end stage aggression will not take p...,"[suffering, end, stage, aggression, will, not,...","[(suffering, VBG), (end, NN), (stage, NN), (ag..."
2,last chemo was today :),just finished my last of 12 abvd infusions. I ...,9.0,"[last, chemo, today, just, finished, last, abv...",last chemo today just finished last abvd infus...,"[last, chemo, today, just, finished, last, abv...","[(last, JJ), (chemo, NN), (today, NN), (just, ..."
3,ER phobia?,Have any of your loved ones checked out of the...,7.0,"[er, phobia, any, ones, checked, hospital, ama...",er phobia any ones checked hospital ama agains...,"[er, phobia, any, ones, checked, hospital, ama...","[(er, RB), (phobia, VBZ), (any, DT), (ones, NN..."
4,"8 minutes into his appointment, another scene ...",At this point it’s just funny to me. I’m not t...,3.0,"[minutes, appointment, scene, made, plus, hosp...",minutes appointment scene made plus hospital h...,"[minutes, appointment, scene, made, plus, hosp...","[(minutes, NNS), (appointment, JJ), (scene, NN..."


In [ ]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [ ]:
df['wordnet_pos'] = df['pos_tags'].apply(lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])
df.head()


,title,body,label,clean_body,clean_body_str,tokenized,pos_tags,wordnet_pos
0,Fuck that Bell!,"Sorry for the title, but I wish the bell ringi...",6.0,"[fuck, bell, sorry, title, wish, bell, ringing...",fuck bell sorry title wish bell ringing party ...,"[fuck, bell, sorry, title, wish, bell, ringing...","[(fuck, NN), (bell, NN), (sorry, JJ), (title, ...","[(fuck, n), (bell, n), (sorry, a), (title, n),..."
1,Father suffering from end stage aggression. Wo...,My dad was diagnosed with stage 4 bone cancer ...,7.0,"[suffering, end, stage, aggression, will not, ...",suffering end stage aggression will not take p...,"[suffering, end, stage, aggression, will, not,...","[(suffering, VBG), (end, NN), (stage, NN), (ag...","[(suffering, v), (end, n), (stage, n), (aggres..."
2,last chemo was today :),just finished my last of 12 abvd infusions. I ...,9.0,"[last, chemo, today, just, finished, last, abv...",last chemo today just finished last abvd infus...,"[last, chemo, today, just, finished, last, abv...","[(last, JJ), (chemo, NN), (today, NN), (just, ...","[(last, a), (chemo, n), (today, n), (just, r),..."
3,ER phobia?,Have any of your loved ones checked out of the...,7.0,"[er, phobia, any, ones, checked, hospital, ama...",er phobia any ones checked hospital ama agains...,"[er, phobia, any, ones, checked, hospital, ama...","[(er, RB), (phobia, VBZ), (any, DT), (ones, NN...","[(er, r), (phobia, v), (any, n), (ones, n), (c..."
4,"8 minutes into his appointment, another scene ...",At this point it’s just funny to me. I’m not t...,3.0,"[minutes, appointment, scene, made, plus, hosp...",minutes appointment scene made plus hospital h...,"[minutes, appointment, scene, made, plus, hosp...","[(minutes, NNS), (appointment, JJ), (scene, NN...","[(minutes, n), (appointment, a), (scene, n), (..."


In [ ]:
#Lemmatization
wnl = WordNetLemmatizer()
df['lemmatized'] = df['wordnet_pos'].apply(lambda x: [wnl.lemmatize(word, tag) for word, tag in x])
df.head()

,title,body,label,clean_body,clean_body_str,tokenized,pos_tags,wordnet_pos,lemmatized
0,Fuck that Bell!,"Sorry for the title, but I wish the bell ringi...",6.0,"[fuck, bell, sorry, title, wish, bell, ringing...",fuck bell sorry title wish bell ringing party ...,"[fuck, bell, sorry, title, wish, bell, ringing...","[(fuck, NN), (bell, NN), (sorry, JJ), (title, ...","[(fuck, n), (bell, n), (sorry, a), (title, n),...","[fuck, bell, sorry, title, wish, bell, ring, p..."
1,Father suffering from end stage aggression. Wo...,My dad was diagnosed with stage 4 bone cancer ...,7.0,"[suffering, end, stage, aggression, will not, ...",suffering end stage aggression will not take p...,"[suffering, end, stage, aggression, will, not,...","[(suffering, VBG), (end, NN), (stage, NN), (ag...","[(suffering, v), (end, n), (stage, n), (aggres...","[suffer, end, stage, aggression, will, not, ta..."
2,last chemo was today :),just finished my last of 12 abvd infusions. I ...,9.0,"[last, chemo, today, just, finished, last, abv...",last chemo today just finished last abvd infus...,"[last, chemo, today, just, finished, last, abv...","[(last, JJ), (chemo, NN), (today, NN), (just, ...","[(last, a), (chemo, n), (today, n), (just, r),...","[last, chemo, today, just, finish, last, abvd,..."
3,ER phobia?,Have any of your loved ones checked out of the...,7.0,"[er, phobia, any, ones, checked, hospital, ama...",er phobia any ones checked hospital ama agains...,"[er, phobia, any, ones, checked, hospital, ama...","[(er, RB), (phobia, VBZ), (any, DT), (ones, NN...","[(er, r), (phobia, v), (any, n), (ones, n), (c...","[er, phobia, any, one, check, hospital, ama, a..."
4,"8 minutes into his appointment, another scene ...",At this point it’s just funny to me. I’m not t...,3.0,"[minutes, appointment, scene, made, plus, hosp...",minutes appointment scene made plus hospital h...,"[minutes, appointment, scene, made, plus, hosp...","[(minutes, NNS), (appointment, JJ), (scene, NN...","[(minutes, n), (appointment, a), (scene, n), (...","[minute, appointment, scene, make, plus, hospi..."


In [ ]:
df.to_pickle("Reddit_dataset_clean.pkl")

In [ ]:
df.to_csv('Reddit_dataset_clean_labelled.csv')